# `Create Database for MIMIC Dataset`

Creates SQLite dataset for MIMIC-IV v3.1 

### Access Data Files
- [ ] file1
- [ ] file2

### Create Databases
- [ ] table1
- [ ] table2

### Read Aquisition Parameters
- [ ] table1





## 1. `Define Data Files`

In [ ]:
# Add project root to sys.path

import sys
from pathlib import Path

ROOT = Path.cwd()
for parent in [ROOT] + list(ROOT.parents):
    if (parent / "config").is_dir():
        ROOT = parent
        break
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print( '[X] Root fount' )

In [ ]:
# Read settings and verify CSV files exist

from xrh.settings import load_settings


settings = load_settings()
if not settings.admission_csv.exists():
    raise FileNotFoundError(f"Admission CSV file not found at {settings.admission_csv}")

if not settings.patient_csv.exists():
    raise FileNotFoundError(f"Patient CSV file not found at {settings.patient_csv}")

if not settings.transfer_csv.exists():
    raise FileNotFoundError(f"Transfer CSV file not found at {settings.transfer_csv}")

if not settings.d_hcpcs_csv.exists():
    raise FileNotFoundError(f"D HCPCS CSV file not found at {settings.d_hcpcs_csv}")

if not settings.d_icd_diagnoses_csv.exists():
    raise FileNotFoundError(f"D ICD Diagnoses CSV file not found at {settings.d_icd_diagnoses_csv}")

if not settings.d_icd_procedures_csv.exists():
    raise FileNotFoundError(f"D ICD Procedures CSV file not found at {settings.d_icd_procedures_csv}")

if not settings.d_labitems_csv.exists():
    raise FileNotFoundError(f"D Lab Items CSV file not found at {settings.d_labitems_csv}")

if not settings.diagnoses_icd_csv.exists():
    raise FileNotFoundError(f"Diagnoses ICD CSV file not found at {settings.diagnoses_icd_csv}")

if not settings.drgcodes_csv.exists():
    raise FileNotFoundError(f"DRG Codes CSV file not found at {settings.drgcodes_csv}")

if not settings.emar_csv.exists():
    raise FileNotFoundError(f"EMAR CSV file not found at {settings.emar_csv}")

if not settings.emar_detail_csv.exists():
    raise FileNotFoundError(f"EMAR Detail CSV file not found at {settings.emar_detail_csv}")

if not settings.hcpcsevents_csv.exists():
    raise FileNotFoundError(f"HCPCS Events CSV file not found at {settings.hcpcsevents_csv}")

if not settings.labevents_csv.exists():
    raise FileNotFoundError(f"Lab Events CSV file not found at {settings.labevents_csv}")

if not settings.microbiologyevents_csv.exists():
    raise FileNotFoundError(f"Microbiology Events CSV file not found at {settings.microbiologyevents_csv}")

if not settings.pharmacy_csv.exists():
    raise FileNotFoundError(f"Pharmacy CSV file not found at {settings.pharmacy_csv}")

if not settings.poe_csv.exists():
    raise FileNotFoundError(f"POE CSV file not found at {settings.poe_csv}")

if not settings.poe_detail_csv.exists():
    raise FileNotFoundError(f"POE Detail CSV file not found at {settings.poe_detail_csv}")

if not settings.prescriptions_csv.exists():
    raise FileNotFoundError(f"Prescriptions CSV file not found at {settings.prescriptions_csv}")

if not settings.procedures_icd_csv.exists():
    raise FileNotFoundError(f"Procedures ICD CSV file not found at {settings.procedures_icd_csv}")

if not settings.services_csv.exists():
    raise FileNotFoundError(f"Services CSV file not found at {settings.services_csv}")

print( '[X] All required CSV files found' )

# 2.1 `Create Databese: HOSP module`



| **Done** | **Table Name**        | **Description** |
|-----------|-----------------------|-----------------|
| [X] | **omr**                | The Online Medical Record (OMR) table contains miscellaneous information from the EHR. |
| [X] | **provider**           | The provider table lists deidentified provider identifiers used in the database. |
| [X] | **admissions**         | Detailed information about hospital stays. |
| [ ] | **d_hcpcs**            | Dimension table for hcpcsevents; provides a description of CPT codes. |
| [ ] | **d_icd_diagnoses**    | Dimension table for diagnoses_icd; provides a description of ICD-9/ICD-10 billed diagnoses. |
| [ ] | **d_icd_procedures**   | Dimension table for procedures_icd; provides a description of ICD-9/ICD-10 billed procedures. |
| [ ] | **d_labitems**         | Dimension table for labevents; provides a description of all lab items. |
| [ ] | **diagnoses_icd**      | Billed ICD-9/ICD-10 diagnoses for hospitalizations. |
| [ ] | **drgcodes**           | Billed diagnosis related group (DRG) codes for hospitalizations. |
| [ ] | **emar**               | The Electronic Medicine Administration Record (eMAR); barcode scanning of medications at the time of administration. |
| [ ] | **emar_detail**        | Supplementary information for electronic administrations recorded in emar. |
| [ ] | **hcpcsevents**        | Billed events occurring during the hospitalization. Includes CPT codes. |
| [ ] | **labevents**          | Laboratory measurements sourced from patient derived specimens. |
| [ ] | **microbiologyevents** | Microbiology cultures. |
| [ ] | **patients**           | Patients' gender, age, and date of death if information exists. |
| [ ] | **pharmacy**           | Formulary, dosing, and other information for prescribed medications. |
| [ ] | **poe**                | Orders made by providers relating to patient care. |
| [ ] | **poe_detail**         | Supplementary information for orders made by providers in the hospital. |
| [ ] | **prescriptions**      | Prescribed medications. |
| [ ] | **procedures_icd**     | Billed procedures for patients during their hospital stay. |
| [ ] | **services**           | The hospital service(s) which cared for the patient during their hospitalization. |
| [ ] | **transfers**          | Detailed information about patients' unit transfers. |


### 2.1.01 `Hosp: OMR`

In [ ]:
# === CREATE TABLE: omr ===
schema_sql_omr = """
DROP TABLE IF EXISTS omr;
CREATE TABLE omr (
    subject_id    INTEGER NOT NULL,
    chartdate     TEXT    NOT NULL,   -- store DATE as ISO-8601 'YYYY-MM-DD'
    seq_num       INTEGER NOT NULL,
    result_name   TEXT    NOT NULL,
    result_value  TEXT    NOT NULL
);
/* Optional indexes (uncomment if useful)
CREATE INDEX IF NOT EXISTS idx_omr_subject ON omr(subject_id);
CREATE INDEX IF NOT EXISTS idx_omr_subject_date ON omr(subject_id, chartdate);
*/
"""

with conn:
    conn.executescript(schema_sql_omr)
print("✅ Table 'omr' created.")

# === LOAD CSV -> omr ===
# Adjust this path if your settings object uses a different attribute name
omr_csv_path = getattr(settings, "omr_csv", None) or csv_path  # fallback to previous csv_path if needed

# Columns expected in the CSV
expected_cols = ["subject_id", "chartdate", "seq_num", "result_name", "result_value"]

# Read CSV; parse chartdate as date if present
try:
    df_omr = pd.read_csv(omr_csv_path, parse_dates=["chartdate"], infer_datetime_format=True, keep_date_col=True)
except ValueError:
    # If 'chartdate' not present or parse error, read without date parsing
    df_omr = pd.read_csv(omr_csv_path)

# If your CSV headers differ, map them here (example shown; edit/remove as needed)
# rename_map = {"SUBJECT_ID": "subject_id", "CHARTDATE": "chartdate", "SEQ_NUM": "seq_num",
#               "RESULT_NAME": "result_name", "RESULT_VALUE": "result_value"}
rename_map = {}
if rename_map:
    df_omr = df_omr.rename(columns=rename_map)

# Validate required columns, enforce order
missing = [c for c in expected_cols if c not in df_omr.columns]
if missing:
    raise ValueError(f"OMR CSV is missing required columns: {missing}")

df_omr = df_omr[expected_cols]

# Format chartdate to 'YYYY-MM-DD' strings (SQLite stores as TEXT)
if "chartdate" in df_omr.columns:
    if pd.api.types.is_datetime64_any_dtype(df_omr["chartdate"]):
        df_omr["chartdate"] = df_omr["chartdate"].dt.strftime("%Y-%m-%d")
    else:
        # If it's already a string, normalize by slicing the date part (optional)
        df_omr["chartdate"] = df_omr["chartdate"].astype(str).str[:10]

# Clean integer columns
for c in ["subject_id", "seq_num"]:
    if c in df_omr.columns:
        df_omr[c] = pd.to_numeric(df_omr[c], errors="coerce").astype("Int64")

# Insert into SQLite
with conn:
    df_omr.to_sql("omr", conn, if_exists="append", index=False)

# Quick verification
rows = conn.execute("SELECT COUNT(*) FROM omr;").fetchone()[0]
print(f"✅ Loaded {rows} rows into 'omr'.")
display(pd.read_sql("SELECT * FROM omr LIMIT 5;", conn))


### 2.1.02 `Hosp: Provider`

In [ ]:
# === CREATE TABLE: provider ===
schema_sql_provider = """
DROP TABLE IF EXISTS provider;
CREATE TABLE provider (
    provider_id TEXT NOT NULL
);
/* Optional index if provider_id is often used for joins */
-- CREATE UNIQUE INDEX IF NOT EXISTS idx_provider_id ON provider(provider_id);
"""

with conn:
    conn.executescript(schema_sql_provider)
print("✅ Table 'provider' created.")
+
# === LOAD CSV -> provider ===
# Adjust CSV path as needed; expects settings.provider_csv to exist
provider_csv_path = getattr(settings, "provider_csv", None)

if provider_csv_path is None:
    raise ValueError("⚠️ Please define settings.provider_csv with the path to your provider CSV file.")

# Read CSV
df_provider = pd.read_csv(provider_csv_path, dtype=str)  # ensure provider_id stays as text

# Normalize column name (handle variations like 'PROVIDER_ID')
rename_map = {"PROVIDER_ID": "provider_id"}
df_provider = df_provider.rename(columns={c: c.lower() for c in df_provider.columns})
df_provider = df_provider.rename(columns=rename_map)

# Validate expected column
if "provider_id" not in df_provider.columns:
    raise ValueError("⚠️ The provider CSV must contain a 'provider_id' column.")

# Keep only that column (avoid extra)
df_provider = df_provider[["provider_id"]]

# Drop duplicates if any
df_provider = df_provider.drop_duplicates()

# Insert into SQLite
with conn:
    df_provider.to_sql("provider", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM provider;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'provider'.")
display(pd.read_sql("SELECT * FROM provider LIMIT 5;", conn))


### 2.1.03 `Hosp: Admission`

In [ ]:
import os
import sqlite3
import pandas as pd

# === CONFIG: set your paths here ===
db_dir   = settings.DB_path.parent                         # folder where the DB will live
csv_path = settings.admission_csv       # <-- change to your CSV path

db_dir.mkdir(parents=True, exist_ok=True)
db_path = settings.DB_path            # database filename

# === 1) Create the database and the admissions table ===
schema_sql = """
DROP TABLE IF EXISTS admissions;
CREATE TABLE admissions (
    subject_id             INTEGER NOT NULL,
    hadm_id                INTEGER NOT NULL,
    admittime              TEXT    NOT NULL,   -- store TIMESTAMP as ISO-8601 text
    dischtime              TEXT,
    deathtime              TEXT,
    admission_type         TEXT    NOT NULL,
    admit_provider_id      TEXT,
    admission_location     TEXT,
    discharge_location     TEXT,
    insurance              TEXT,
    language               TEXT,
    marital_status         TEXT,
    race                   TEXT,
    edregtime              TEXT,
    edouttime              TEXT,
    hospital_expire_flag   INTEGER
);
-- Optional helpful indexes (uncomment if you want them)
-- CREATE INDEX idx_admissions_hadm ON admissions(hadm_id);
-- CREATE INDEX idx_admissions_subject ON admissions(subject_id);
"""

conn = sqlite3.connect(db_path)
with conn:
    conn.executescript(schema_sql)

print(f"✅ Created database at: {db_path.resolve()}")
print("✅ Created table: admissions")

# === 2) Load CSV and insert into admissions ===
# If your CSV columns already match exactly, this will just work.
# Otherwise, you can rename columns via the 'rename_map' below.
parse_date_cols = [
    "admittime", "dischtime", "deathtime", "edregtime", "edouttime"
]

# Read CSV; if your timestamps are ISO-8601 already, you can set dtype=str instead of parse_dates
try:
    df = pd.read_csv(csv_path, parse_dates=parse_date_cols, infer_datetime_format=True, keep_date_col=True)
except ValueError:
    # Fall back if some timestamp cols are missing in the CSV
    df = pd.read_csv(csv_path)

# Ensure column names match the target table
expected_cols = [
    "subject_id","hadm_id","admittime","dischtime","deathtime",
    "admission_type","admit_provider_id","admission_location","discharge_location",
    "insurance","language","marital_status","race","edregtime","edouttime",
    "hospital_expire_flag"
]

# If your CSV headers differ, map them here, e.g. {'SUBJECT_ID':'subject_id', 'HADM_ID':'hadm_id', ...}
rename_map = {}
if rename_map:
    df = df.rename(columns=rename_map)

# Keep only expected columns (and in the right order)
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"The CSV is missing required columns: {missing}")

df = df[expected_cols]

# Convert datetime columns to ISO 8601 strings for SQLite (TEXT)
for c in ["admittime","dischtime","deathtime","edregtime","edouttime"]:
    if c in df.columns:
        # Convert to string only where not null
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            df[c] = df[c].dt.strftime("%Y-%m-%d %H:%M:%S")
        else:
            # If already strings, leave as-is
            pass

# Enforce integer types where appropriate (SQLite is flexible, but this helps cleanliness)
int_cols = ["subject_id","hadm_id","hospital_expire_flag"]
for c in int_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

# Insert into the table
# We defined the table explicitly, so we APPEND.
with conn:
    df.to_sql("admissions", conn, if_exists="append", index=False)

# Quick verification
row_count = conn.execute("SELECT COUNT(*) FROM admissions").fetchone()[0]
print(f"✅ Inserted {row_count} rows into admissions")

# Peek at a few rows
preview = pd.read_sql("SELECT * FROM admissions LIMIT 5;", conn)
conn.close()
preview


### 2.1.04 `Hosp: d_hcpcs`

In [ ]:
# === CREATE TABLE: d_hcpcs ===
schema_sql_d_hcpcs = """
DROP TABLE IF EXISTS d_hcpcs;
CREATE TABLE d_hcpcs (
    code               TEXT NOT NULL,   -- CHAR(5) in Postgres, stored as TEXT in SQLite
    category           INTEGER,
    long_description   TEXT,
    short_description  TEXT
);
/* Optional index if codes are used for lookups */
-- CREATE UNIQUE INDEX IF NOT EXISTS idx_d_hcpcs_code ON d_hcpcs(code);
"""

with conn:
    conn.executescript(schema_sql_d_hcpcs)
print("✅ Table 'd_hcpcs' created.")

# === LOAD CSV -> d_hcpcs ===
d_hcpcs_csv_path = getattr(settings, "d_hcpcs_csv", None)
if d_hcpcs_csv_path is None:
    raise ValueError("⚠️ Please define settings.d_hcpcs_csv with the path to your d_hcpcs CSV file.")

# Read CSV (always treat code as string)
df_d_hcpcs = pd.read_csv(d_hcpcs_csv_path, dtype=str)

# Normalize column names (handle uppercase/lowercase mismatches)
rename_map = {
    "CODE": "code",
    "CATEGORY": "category",
    "LONG_DESCRIPTION": "long_description",
    "SHORT_DESCRIPTION": "short_description"
}
df_d_hcpcs = df_d_hcpcs.rename(columns={c: c.lower() for c in df_d_hcpcs.columns})
df_d_hcpcs = df_d_hcpcs.rename(columns=rename_map)

# Verify required columns
expected_cols = ["code", "category", "long_description", "short_description"]
missing = [c for c in expected_cols if c not in df_d_hcpcs.columns]
if missing:
    raise ValueError(f"⚠️ d_hcpcs CSV is missing required columns: {missing}")

# Keep only expected columns and enforce types
df_d_hcpcs = df_d_hcpcs[expected_cols]

# Convert category to integer where possible
df_d_hcpcs["category"] = pd.to_numeric(df_d_hcpcs["category"], errors="coerce").astype("Int64")

# Drop duplicate codes if any
df_d_hcpcs = df_d_hcpcs.drop_duplicates(subset=["code"])

# Insert into SQLite
with conn:
    df_d_hcpcs.to_sql("d_hcpcs", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM d_hcpcs;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'd_hcpcs'.")
display(pd.read_sql("SELECT * FROM d_hcpcs LIMIT 5;", conn))


### 2.1.05 `Hosp: d_icd_diagnoses`

In [ ]:
# === CREATE TABLE: d_icd_diagnoses ===
schema_sql_d_icd_diagnoses = """
DROP TABLE IF EXISTS d_icd_diagnoses;
CREATE TABLE d_icd_diagnoses (
    icd_code     TEXT    NOT NULL,   -- CHAR(7) in Postgres → TEXT in SQLite
    icd_version  INTEGER NOT NULL,
    long_title   TEXT
);
/* Optional index for faster ICD code lookups */
-- CREATE UNIQUE INDEX IF NOT EXISTS idx_d_icd_diagnoses_code ON d_icd_diagnoses(icd_code, icd_version);
"""

with conn:
    conn.executescript(schema_sql_d_icd_diagnoses)
print("✅ Table 'd_icd_diagnoses' created.")

# === LOAD CSV -> d_icd_diagnoses ===
d_icd_diagnoses_csv_path = getattr(settings, "d_icd_diagnoses_csv", None)
if d_icd_diagnoses_csv_path is None:
    raise ValueError("⚠️ Please define settings.d_icd_diagnoses_csv with the path to your d_icd_diagnoses CSV file.")

# Read CSV (keep all as strings first)
df_d_icd_diagnoses = pd.read_csv(d_icd_diagnoses_csv_path, dtype=str)

# Normalize column names (handle uppercase/lowercase mismatches)
rename_map = {
    "ICD_CODE": "icd_code",
    "ICD_VERSION": "icd_version",
    "LONG_TITLE": "long_title"
}
df_d_icd_diagnoses = df_d_icd_diagnoses.rename(columns={c: c.lower() for c in df_d_icd_diagnoses.columns})
df_d_icd_diagnoses = df_d_icd_diagnoses.rename(columns=rename_map)

# Validate required columns
expected_cols = ["icd_code", "icd_version", "long_title"]
missing = [c for c in expected_cols if c not in df_d_icd_diagnoses.columns]
if missing:
    raise ValueError(f"⚠️ d_icd_diagnoses CSV is missing required columns: {missing}")

# Keep only expected columns in order
df_d_icd_diagnoses = df_d_icd_diagnoses[expected_cols]

# Convert icd_version to integer
df_d_icd_diagnoses["icd_version"] = pd.to_numeric(df_d_icd_diagnoses["icd_version"], errors="coerce").astype("Int64")

# Drop duplicates (by code & version)
df_d_icd_diagnoses = df_d_icd_diagnoses.drop_duplicates(subset=["icd_code", "icd_version"])

# Insert into SQLite
with conn:
    df_d_icd_diagnoses.to_sql("d_icd_diagnoses", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM d_icd_diagnoses;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'd_icd_diagnoses'.")
display(pd.read_sql("SELECT * FROM d_icd_diagnoses LIMIT 5;", conn))


### 2.1.06 `Hosp: d_icd_procedures`

In [ ]:
# === CREATE TABLE: d_icd_procedures ===
schema_sql_d_icd_procedures = """
DROP TABLE IF EXISTS d_icd_procedures;
CREATE TABLE d_icd_procedures (
    icd_code     TEXT    NOT NULL,   -- CHAR(7) in Postgres → TEXT in SQLite
    icd_version  INTEGER NOT NULL,
    long_title   TEXT
);
/* Optional index for faster lookups */
-- CREATE UNIQUE INDEX IF NOT EXISTS idx_d_icd_procedures_code ON d_icd_procedures(icd_code, icd_version);
"""

with conn:
    conn.executescript(schema_sql_d_icd_procedures)
print("✅ Table 'd_icd_procedures' created.")

# === LOAD CSV -> d_icd_procedures ===
d_icd_procedures_csv_path = getattr(settings, "d_icd_procedures_csv", None)
if d_icd_procedures_csv_path is None:
    raise ValueError("⚠️ Please define settings.d_icd_procedures_csv with the path to your d_icd_procedures CSV file.")

# Read CSV (keep everything as text initially)
df_d_icd_procedures = pd.read_csv(d_icd_procedures_csv_path, dtype=str)

# Normalize and rename columns to match schema
rename_map = {
    "ICD_CODE": "icd_code",
    "ICD_VERSION": "icd_version",
    "LONG_TITLE": "long_title"
}
df_d_icd_procedures = df_d_icd_procedures.rename(columns={c: c.lower() for c in df_d_icd_procedures.columns})
df_d_icd_procedures = df_d_icd_procedures.rename(columns=rename_map)

# Verify columns
expected_cols = ["icd_code", "icd_version", "long_title"]
missing = [c for c in expected_cols if c not in df_d_icd_procedures.columns]
if missing:
    raise ValueError(f"⚠️ d_icd_procedures CSV is missing required columns: {missing}")

# Keep only expected columns
df_d_icd_procedures = df_d_icd_procedures[expected_cols]

# Convert icd_version to integer (nullable-safe)
df_d_icd_procedures["icd_version"] = pd.to_numeric(df_d_icd_procedures["icd_version"], errors="coerce").astype("Int64")

# Drop duplicates by (icd_code, icd_version)
df_d_icd_procedures = df_d_icd_procedures.drop_duplicates(subset=["icd_code", "icd_version"])

# Insert into SQLite
with conn:
    df_d_icd_procedures.to_sql("d_icd_procedures", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM d_icd_procedures;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'd_icd_procedures'.")
display(pd.read_sql("SELECT * FROM d_icd_procedures LIMIT 5;", conn))


### 2.1.07 `Hosp: d_labitems`

In [ ]:
# === CREATE TABLE: d_labitems ===
schema_sql_d_labitems = """
DROP TABLE IF EXISTS d_labitems;
CREATE TABLE d_labitems (
    itemid    INTEGER,
    label     TEXT,
    fluid     TEXT,
    category  TEXT
);
/* Optional index for fast lookup by itemid */
-- CREATE UNIQUE INDEX IF NOT EXISTS idx_d_labitems_itemid ON d_labitems(itemid);
"""

with conn:
    conn.executescript(schema_sql_d_labitems)
print("✅ Table 'd_labitems' created.")

# === LOAD CSV -> d_labitems ===
d_labitems_csv_path = getattr(settings, "d_labitems_csv", None)
if d_labitems_csv_path is None:
    raise ValueError("⚠️ Please define settings.d_labitems_csv with the path to your d_labitems CSV file.")

# Read CSV
df_d_labitems = pd.read_csv(d_labitems_csv_path, dtype=str)  # start as string to avoid parsing issues

# Normalize column names (handle uppercase/lowercase differences)
rename_map = {
    "ITEMID": "itemid",
    "LABEL": "label",
    "FLUID": "fluid",
    "CATEGORY": "category"
}
df_d_labitems = df_d_labitems.rename(columns={c: c.lower() for c in df_d_labitems.columns})
df_d_labitems = df_d_labitems.rename(columns=rename_map)

# Verify required columns
expected_cols = ["itemid", "label", "fluid", "category"]
missing = [c for c in expected_cols if c not in df_d_labitems.columns]
if missing:
    raise ValueError(f"⚠️ d_labitems CSV is missing required columns: {missing}")

# Keep only expected columns
df_d_labitems = df_d_labitems[expected_cols]

# Convert itemid to integer (nullable-safe)
df_d_labitems["itemid"] = pd.to_numeric(df_d_labitems["itemid"], errors="coerce").astype("Int64")

# Drop duplicates (by itemid if present)
df_d_labitems = df_d_labitems.drop_duplicates(subset=["itemid"])

# Insert into SQLite
with conn:
    df_d_labitems.to_sql("d_labitems", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM d_labitems;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'd_labitems'.")
display(pd.read_sql("SELECT * FROM d_labitems LIMIT 5;", conn))


### 2.1.08 `Hosp: diagnoses_icd`

In [ ]:
# === CREATE TABLE: diagnoses_icd ===
schema_sql_diagnoses_icd = """
DROP TABLE IF EXISTS diagnoses_icd;
CREATE TABLE diagnoses_icd (
    subject_id   INTEGER NOT NULL,
    hadm_id      INTEGER NOT NULL,
    seq_num      INTEGER NOT NULL,
    icd_code     TEXT,
    icd_version  INTEGER
);
/* Optional indexes to speed up lookups */
-- CREATE INDEX IF NOT EXISTS idx_diagnoses_subject ON diagnoses_icd(subject_id);
-- CREATE INDEX IF NOT EXISTS idx_diagnoses_hadm ON diagnoses_icd(hadm_id);
-- CREATE INDEX IF NOT EXISTS idx_diagnoses_code ON diagnoses_icd(icd_code);
"""

with conn:
    conn.executescript(schema_sql_diagnoses_icd)
print("✅ Table 'diagnoses_icd' created.")

# === LOAD CSV -> diagnoses_icd ===
diagnoses_icd_csv_path = getattr(settings, "diagnoses_icd_csv", None)
if diagnoses_icd_csv_path is None:
    raise ValueError("⚠️ Please define settings.diagnoses_icd_csv with the path to your diagnoses_icd CSV file.")

# Read CSV (treat all as strings first)
df_diagnoses_icd = pd.read_csv(diagnoses_icd_csv_path, dtype=str)

# Normalize column names (handle upper/lowercase differences)
rename_map = {
    "SUBJECT_ID": "subject_id",
    "HADM_ID": "hadm_id",
    "SEQ_NUM": "seq_num",
    "ICD_CODE": "icd_code",
    "ICD_VERSION": "icd_version"
}
df_diagnoses_icd = df_diagnoses_icd.rename(columns={c: c.lower() for c in df_diagnoses_icd.columns})
df_diagnoses_icd = df_diagnoses_icd.rename(columns=rename_map)

# Validate required columns
expected_cols = ["subject_id", "hadm_id", "seq_num", "icd_code", "icd_version"]
missing = [c for c in expected_cols if c not in df_diagnoses_icd.columns]
if missing:
    raise ValueError(f"⚠️ diagnoses_icd CSV is missing required columns: {missing}")

# Keep only expected columns
df_diagnoses_icd = df_diagnoses_icd[expected_cols]

# Convert numeric columns safely
for col in ["subject_id", "hadm_id", "seq_num", "icd_version"]:
    df_diagnoses_icd[col] = pd.to_numeric(df_diagnoses_icd[col], errors="coerce").astype("Int64")

# Drop duplicates (if any)
df_diagnoses_icd = df_diagnoses_icd.drop_duplicates(subset=["subject_id", "hadm_id", "seq_num"])

# Insert into SQLite
with conn:
    df_diagnoses_icd.to_sql("diagnoses_icd", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM diagnoses_icd;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'diagnoses_icd'.")
display(pd.read_sql("SELECT * FROM diagnoses_icd LIMIT 5;", conn))


### 2.1.09 `Hosp: drgcodes`

In [ ]:
# === CREATE TABLE: drgcodes ===
schema_sql_drgcodes = """
DROP TABLE IF EXISTS drgcodes;
CREATE TABLE drgcodes (
    subject_id     INTEGER,
    hadm_id        INTEGER,
    drg_type       TEXT,
    drg_code       TEXT,
    description    TEXT,
    drg_severity   INTEGER,
    drg_mortality  INTEGER
);
/* Optional indexes for faster joins or lookups */
-- CREATE INDEX IF NOT EXISTS idx_drgcodes_subject ON drgcodes(subject_id);
-- CREATE INDEX IF NOT EXISTS idx_drgcodes_hadm ON drgcodes(hadm_id);
-- CREATE INDEX IF NOT EXISTS idx_drgcodes_code ON drgcodes(drg_code);
"""

with conn:
    conn.executescript(schema_sql_drgcodes)
print("✅ Table 'drgcodes' created.")

# === LOAD CSV -> drgcodes ===
drgcodes_csv_path = getattr(settings, "drgcodes_csv", None)
if drgcodes_csv_path is None:
    raise ValueError("⚠️ Please define settings.drgcodes_csv with the path to your drgcodes CSV file.")

# Read CSV as strings (safer for mixed types)
df_drgcodes = pd.read_csv(drgcodes_csv_path, dtype=str)

# Normalize and rename columns
rename_map = {
    "SUBJECT_ID": "subject_id",
    "HADM_ID": "hadm_id",
    "DRG_TYPE": "drg_type",
    "DRG_CODE": "drg_code",
    "DESCRIPTION": "description",
    "DRG_SEVERITY": "drg_severity",
    "DRG_MORTALITY": "drg_mortality"
}
df_drgcodes = df_drgcodes.rename(columns={c: c.lower() for c in df_drgcodes.columns})
df_drgcodes = df_drgcodes.rename(columns=rename_map)

# Validate required columns
expected_cols = ["subject_id", "hadm_id", "drg_type", "drg_code", "description", "drg_severity", "drg_mortality"]
missing = [c for c in expected_cols if c not in df_drgcodes.columns]
if missing:
    raise ValueError(f"⚠️ drgcodes CSV is missing required columns: {missing}")

# Keep only expected columns
df_drgcodes = df_drgcodes[expected_cols]

# Convert integer columns safely
for col in ["subject_id", "hadm_id", "drg_severity", "drg_mortality"]:
    df_drgcodes[col] = pd.to_numeric(df_drgcodes[col], errors="coerce").astype("Int64")

# Drop duplicates
df_drgcodes = df_drgcodes.drop_duplicates(subset=["subject_id", "hadm_id", "drg_code"])

# Insert into SQLite
with conn:
    df_drgcodes.to_sql("drgcodes", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM drgcodes;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'drgcodes'.")
display(pd.read_sql("SELECT * FROM drgcodes LIMIT 5;", conn))


### 2.1.10 `Hosp: emar`

In [ ]:
# === CREATE TABLE: emar ===
schema_sql_emar = """
DROP TABLE IF EXISTS emar;
CREATE TABLE emar (
    subject_id          INTEGER NOT NULL,
    hadm_id             INTEGER,
    emar_id             TEXT    NOT NULL,
    emar_seq            INTEGER NOT NULL,
    poe_id              TEXT    NOT NULL,
    pharmacy_id         INTEGER,
    enter_provider_id   TEXT,
    charttime           TEXT    NOT NULL,  -- store TIMESTAMP as ISO8601 text
    medication          TEXT,
    event_txt           TEXT,
    scheduletime        TEXT,
    storetime           TEXT    NOT NULL
);
/* Optional indexes for faster lookups */
-- CREATE INDEX IF NOT EXISTS idx_emar_subject ON emar(subject_id);
-- CREATE INDEX IF NOT EXISTS idx_emar_hadm ON emar(hadm_id);
-- CREATE INDEX IF NOT EXISTS idx_emar_charttime ON emar(charttime);
"""

with conn:
    conn.executescript(schema_sql_emar)
print("✅ Table 'emar' created.")

# === LOAD CSV -> emar ===
emar_csv_path = getattr(settings, "emar_csv", None)
if emar_csv_path is None:
    raise ValueError("⚠️ Please define settings.emar_csv with the path to your emar CSV file.")

# Read CSV as strings (timestamps converted later)
df_emar = pd.read_csv(emar_csv_path, dtype=str)

# Normalize and rename columns to lowercase
rename_map = {
    "SUBJECT_ID": "subject_id",
    "HADM_ID": "hadm_id",
    "EMAR_ID": "emar_id",
    "EMAR_SEQ": "emar_seq",
    "POE_ID": "poe_id",
    "PHARMACY_ID": "pharmacy_id",
    "ENTER_PROVIDER_ID": "enter_provider_id",
    "CHARTTIME": "charttime",
    "MEDICATION": "medication",
    "EVENT_TXT": "event_txt",
    "SCHEDULETIME": "scheduletime",
    "STORETIME": "storetime"
}
df_emar = df_emar.rename(columns={c: c.lower() for c in df_emar.columns})
df_emar = df_emar.rename(columns=rename_map)

# Verify required columns
expected_cols = [
    "subject_id", "hadm_id", "emar_id", "emar_seq", "poe_id", "pharmacy_id",
    "enter_provider_id", "charttime", "medication", "event_txt",
    "scheduletime", "storetime"
]
missing = [c for c in expected_cols if c not in df_emar.columns]
if missing:
    raise ValueError(f"⚠️ emar CSV is missing required columns: {missing}")

# Keep only expected columns
df_emar = df_emar[expected_cols]

# Convert numeric columns safely
for col in ["subject_id", "hadm_id", "emar_seq", "pharmacy_id"]:
    df_emar[col] = pd.to_numeric(df_emar[col], errors="coerce").astype("Int64")

# Convert timestamps to ISO 8601 text
for col in ["charttime", "scheduletime", "storetime"]:
    if col in df_emar.columns:
        df_emar[col] = pd.to_datetime(df_emar[col], errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")

# Drop duplicates if any
df_emar = df_emar.drop_duplicates(subset=["emar_id", "emar_seq"])

# Insert into SQLite
with conn:
    df_emar.to_sql("emar", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM emar;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'emar'.")
display(pd.read_sql("SELECT * FROM emar LIMIT 5;", conn))


### 2.1.11 `Hosp: emar_detail`

In [ ]:
# === CREATE TABLE: emar_detail ===
schema_sql_emar_detail = """
DROP TABLE IF EXISTS emar_detail;
CREATE TABLE emar_detail (
    subject_id                               INTEGER NOT NULL,
    emar_id                                  TEXT    NOT NULL,   -- VARCHAR(25)
    emar_seq                                 INTEGER NOT NULL,
    parent_field_ordinal                     TEXT,               -- VARCHAR(10)
    administration_type                      TEXT,               -- VARCHAR(50)
    pharmacy_id                              INTEGER,
    barcode_type                             TEXT,               -- VARCHAR(4)
    reason_for_no_barcode                    TEXT,
    complete_dose_not_given                  TEXT,               -- VARCHAR(5)
    dose_due                                 TEXT,               -- VARCHAR(100)
    dose_due_unit                            TEXT,               -- VARCHAR(50)
    dose_given                               TEXT,               -- VARCHAR(255)
    dose_given_unit                          TEXT,               -- VARCHAR(50)
    will_remainder_of_dose_be_given          TEXT,               -- VARCHAR(5)
    product_amount_given                     TEXT,               -- VARCHAR(30)
    product_unit                             TEXT,               -- VARCHAR(30)
    product_code                             TEXT,               -- VARCHAR(30)
    product_description                      TEXT,               -- VARCHAR(255)
    product_description_other                TEXT,               -- VARCHAR(255)
    prior_infusion_rate                      TEXT,               -- VARCHAR(40)
    infusion_rate                            TEXT,               -- VARCHAR(40)
    infusion_rate_adjustment                 TEXT,               -- VARCHAR(50)
    infusion_rate_adjustment_amount          TEXT,               -- VARCHAR(30)
    infusion_rate_unit                       TEXT,               -- VARCHAR(30)
    route                                    TEXT,               -- VARCHAR(10)
    infusion_complete                        TEXT,               -- VARCHAR(1)
    completion_interval                      TEXT,               -- VARCHAR(50)
    new_iv_bag_hung                          TEXT,               -- VARCHAR(1)
    continued_infusion_in_other_location     TEXT,               -- VARCHAR(1)
    restart_interval                         TEXT,
    side                                     TEXT,               -- VARCHAR(10)
    site                                     TEXT,               -- VARCHAR(255)
    non_formulary_visual_verification        TEXT                -- VARCHAR(1)
);
/* Optional indexes */
-- CREATE INDEX IF NOT EXISTS idx_emar_detail_subject ON emar_detail(subject_id);
-- CREATE INDEX IF NOT EXISTS idx_emar_detail_emar ON emar_detail(emar_id, emar_seq);
"""

with conn:
    conn.executescript(schema_sql_emar_detail)
print("✅ Table 'emar_detail' created.")

# === LOAD CSV -> emar_detail ===
emar_detail_csv_path = getattr(settings, "emar_detail_csv", None)
if emar_detail_csv_path is None:
    raise ValueError("⚠️ Please define settings.emar_detail_csv with the path to your emar_detail CSV file.")

import pandas as pd

df = pd.read_csv(emar_detail_csv_path, dtype=str)

# Normalize/rename columns to expected names
rename_map = {
    "SUBJECT_ID": "subject_id",
    "EMAR_ID": "emar_id",
    "EMAR_SEQ": "emar_seq",
    "PARENT_FIELD_ORDINAL": "parent_field_ordinal",
    "ADMINISTRATION_TYPE": "administration_type",
    "PHARMACY_ID": "pharmacy_id",
    "BARCODE_TYPE": "barcode_type",
    "REASON_FOR_NO_BARCODE": "reason_for_no_barcode",
    "COMPLETE_DOSE_NOT_GIVEN": "complete_dose_not_given",
    "DOSE_DUE": "dose_due",
    "DOSE_DUE_UNIT": "dose_due_unit",
    "DOSE_GIVEN": "dose_given",
    "DOSE_GIVEN_UNIT": "dose_given_unit",
    "WILL_REMAINDER_OF_DOSE_BE_GIVEN": "will_remainder_of_dose_be_given",
    "PRODUCT_AMOUNT_GIVEN": "product_amount_given",
    "PRODUCT_UNIT": "product_unit",
    "PRODUCT_CODE": "product_code",
    "PRODUCT_DESCRIPTION": "product_description",
    "PRODUCT_DESCRIPTION_OTHER": "product_description_other",
    "PRIOR_INFUSION_RATE": "prior_infusion_rate",
    "INFUSION_RATE": "infusion_rate",
    "INFUSION_RATE_ADJUSTMENT": "infusion_rate_adjustment",
    "INFUSION_RATE_ADJUSTMENT_AMOUNT": "infusion_rate_adjustment_amount",
    "INFUSION_RATE_UNIT": "infusion_rate_unit",
    "ROUTE": "route",
    "INFUSION_COMPLETE": "infusion_complete",
    "COMPLETION_INTERVAL": "completion_interval",
    "NEW_IV_BAG_HUNG": "new_iv_bag_hung",
    "CONTINUED_INFUSION_IN_OTHER_LOCATION": "continued_infusion_in_other_location",
    "RESTART_INTERVAL": "restart_interval",
    "SIDE": "side",
    "SITE": "site",
    "NON_FORMULARY_VISUAL_VERIFICATION": "non_formulary_visual_verification",
}
df = df.rename(columns={c: c.lower() for c in df.columns}).rename(columns=rename_map)

expected_cols = [
    "subject_id","emar_id","emar_seq","parent_field_ordinal","administration_type",
    "pharmacy_id","barcode_type","reason_for_no_barcode","complete_dose_not_given",
    "dose_due","dose_due_unit","dose_given","dose_given_unit","will_remainder_of_dose_be_given",
    "product_amount_given","product_unit","product_code","product_description",
    "product_description_other","prior_infusion_rate","infusion_rate","infusion_rate_adjustment",
    "infusion_rate_adjustment_amount","infusion_rate_unit","route","infusion_complete",
    "completion_interval","new_iv_bag_hung","continued_infusion_in_other_location",
    "restart_interval","side","site","non_formulary_visual_verification"
]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"⚠️ emar_detail CSV is missing required columns: {missing}")

df = df[expected_cols]

# Cast ints safely
for col in ["subject_id", "emar_seq", "pharmacy_id"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# Deduplicate (natural key)
df = df.drop_duplicates(subset=["emar_id", "emar_seq", "parent_field_ordinal"])

with conn:
    df.to_sql("emar_detail", conn, if_exists="append", index=False)

count = conn.execute("SELECT COUNT(*) FROM emar_detail;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'emar_detail'.")
display(pd.read_sql("SELECT * FROM emar_detail LIMIT 5;", conn))


### 2.1.12 `Hosp: hcpcsevents`

In [ ]:
# === CREATE TABLE: hcpcsevents ===
schema_sql_hcpcsevents = """
DROP TABLE IF EXISTS hcpcsevents;
CREATE TABLE hcpcsevents (
    subject_id        INTEGER NOT NULL,
    hadm_id           INTEGER NOT NULL,
    chartdate         TEXT,          -- DATE stored as 'YYYY-MM-DD'
    hcpcs_cd          TEXT    NOT NULL,   -- CHAR(5)
    seq_num           INTEGER NOT NULL,
    short_description TEXT
);
/* Optional indexes */
-- CREATE INDEX IF NOT EXISTS idx_hcpcs_subject ON hcpcsevents(subject_id);
-- CREATE INDEX IF NOT EXISTS idx_hcpcs_hadm    ON hcpcsevents(hadm_id);
-- CREATE INDEX IF NOT EXISTS idx_hcpcs_code    ON hcpcsevents(hcpcs_cd);
"""

with conn:
    conn.executescript(schema_sql_hcpcsevents)
print("✅ Table 'hcpcsevents' created.")

# === LOAD CSV -> hcpcsevents ===
hcpcsevents_csv_path = getattr(settings, "hcpcsevents_csv", None)
if hcpcsevents_csv_path is None:
    raise ValueError("⚠️ Please define settings.hcpcsevents_csv with the path to your HCPCS events CSV file.")

import pandas as pd

# Read as strings first (we'll parse date next)
df_hcpcs = pd.read_csv(hcpcsevents_csv_path, dtype=str)

# Normalize/rename columns to match schema
rename_map = {
    "SUBJECT_ID": "subject_id",
    "HADM_ID": "hadm_id",
    "CHARTDATE": "chartdate",
    "HCPCS_CD": "hcpcs_cd",
    "SEQ_NUM": "seq_num",
    "SHORT_DESCRIPTION": "short_description",
}
df_hcpcs = df_hcpcs.rename(columns={c: c.lower() for c in df_hcpcs.columns}).rename(columns=rename_map)

expected_cols = ["subject_id", "hadm_id", "chartdate", "hcpcs_cd", "seq_num", "short_description"]
missing = [c for c in expected_cols if c not in df_hcpcs.columns]
if missing:
    raise ValueError(f"⚠️ hcpcsevents CSV is missing required columns: {missing}")

# Keep only expected columns (and in order)
df_hcpcs = df_hcpcs[expected_cols]

# Parse chartdate to ISO-8601 date (YYYY-MM-DD) text
if "chartdate" in df_hcpcs.columns:
    parsed = pd.to_datetime(df_hcpcs["chartdate"], errors="coerce").dt.strftime("%Y-%m-%d")
    df_hcpcs["chartdate"] = parsed

# Cast integer-like fields
for col in ["subject_id", "hadm_id", "seq_num"]:
    df_hcpcs[col] = pd.to_numeric(df_hcpcs[col], errors="coerce").astype("Int64")

# Optional: drop duplicates by (subject_id, hadm_id, hcpcs_cd, seq_num)
df_hcpcs = df_hcpcs.drop_duplicates(subset=["subject_id", "hadm_id", "hcpcs_cd", "seq_num"])

# Insert into SQLite
with conn:
    df_hcpcs.to_sql("hcpcsevents", conn, if_exists="append", index=False)

# Quick verification
count = conn.execute("SELECT COUNT(*) FROM hcpcsevents;").fetchone()[0]
print(f"✅ Loaded {count} rows into 'hcpcsevents'.")
display(pd.read_sql("SELECT * FROM hcpcsevents LIMIT 5;", conn))


### 2.1.13 `Hosp: labevents`

### 2.1.14 `Hosp: microbiologyevents`

### 2.1.15 `Hosp: patients`

### 2.1.16 `Hosp: pharmacy`

### 2.1.17 `Hosp: poe`

### 2.1.18 `Hosp: poe_detail`

### 2.1.19 `Hosp: prescriptions`

### 2.1.20 `Hosp: procedures_icd`

### 2.1.21 `Hosp: services`

### 2.1.22 `Hosp: transfers`

# 2.2 `Create Databese: ICU module`



### 2.2.01 `ICU: `

### 2.2.02 `ICU: `

### 2.2.03 `ICU: `